In [1]:
import numpy as np
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.util import pr
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')


In [5]:
data=pd.read_csv('/kaggle/input/dynamically-generated-hate-speech-dataset/2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv')


In [6]:
data.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [7]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
stemmer = nltk.SnowballStemmer("english")
 
# Create WordNetLemmatizer object
lm= nltk.WordNetLemmatizer()

In [10]:
#remove columns we do not need 
cl=['type','model_wrong','db.model_preds','status','round','split','annotator','Unnamed: 0','id']
data.drop(columns=cl,inplace=True)

In [11]:
#check null values
data.isnull().sum()

text     0
label    0
dtype: int64

In [6]:
#remove null values
#data.dropna(axis = 0,inplace = True)

In [12]:
def clean(text):
 text=str(text)
  #remove hyperlinks
 text = re.sub('https?://\S+|www\.\S+', '', text)
 #remove any character enter < > and < >
 text = re.sub('<.*?>+', '', text)
 #remove any character enter [ ] and []
 text = re.sub('<.*?>+', '', text)
 #remove punctuation
 text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
 #remove \n newline
 text = re.sub('\n', '', text)
 #remove symbols and digital(a9a)
 text = re.sub('\w*\d\w*', '', text)
 #stop word
 text = [word for word in text.split(' ') if word not in stop_words]
 text=" ".join(text)
 #lemmatization
 text = [lm.lemmatize(word) for word in text.split(' ')]
 text=" ".join(text)
 return text
data["clean_sentence_training"]=data["text"].apply(clean)
data.head()

,text,label,clean_sentence_training
0,It should be legal for a husband to rape his wife,hate,It legal husband rape wife
1,dalits are lowlife scum,hate,dalits lowlife scum
2,dalits are lowlives,hate,dalits lowlives
3,It was a better world when women dare not ques...,hate,It better world woman dare question men
4,I don't work this hard so that those immigrant...,hate,I dont work hard immigrant take benefit


In [14]:
#rename text column to raw_sentence
data.rename(columns={'text':'raw_sentence'},inplace=True)

In [15]:
#create corpus_name
data['corpus_name']= 'Dynamically Generated Hate Speech'

In [16]:
#reorder the columns 
order=[3,0,1,2]
df=data[[data.columns[i] for  i in order]]
df.head()

,corpus_name,raw_sentence,label,clean_sentence_training
0,Dynamically Generated Hate Speech,It should be legal for a husband to rape his wife,hate,It legal husband rape wife
1,Dynamically Generated Hate Speech,dalits are lowlife scum,hate,dalits lowlife scum
2,Dynamically Generated Hate Speech,dalits are lowlives,hate,dalits lowlives
3,Dynamically Generated Hate Speech,It was a better world when women dare not ques...,hate,It better world woman dare question men
4,Dynamically Generated Hate Speech,I don't work this hard so that those immigrant...,hate,I dont work hard immigrant take benefit


In [17]:
#encoding the label feature
df['label']=df['label'].replace({'hate':2,'nohate':0})

In [18]:
#emojis
!pip install advertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 kB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.9/272.9 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 7.0 MB/s eta 0:00:00
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=71137a769a55909a81de65ab87caf7d78927772adf1dbe97eed9ad46aa350b67
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


In [21]:
#emojis
import advertools as adv

In [23]:
#Extract emojis
emoji_summary = adv.extract_emoji(df['clean_sentence_training'])
#emoji_summary['overview']

In [24]:
#summary about emojis
emoji_summary['overview']

{'num_posts': 40623,
 'num_emoji': 12,
 'emoji_per_post': 0.0002953991581123994,
 'unique_emoji': 2}

In [25]:
#listed emojis
emoji_summary['emoji_flat']

['™', '™', '™', '™', '™', '®', '®', '™', '™', '™', '™', '®']

In [28]:
#remove emojis
df['clean_sentence_EDA']=(df['clean_sentence_training'].str.replace('[^\w\s\#@/:%.,_-]','',flags=re.UNICODE))

In [29]:
#check emojis after remove them
emoji_summary = adv.extract_emoji(df['clean_sentence_EDA'])
emoji_summary['overview']

{'num_posts': 40623, 'num_emoji': 0, 'emoji_per_post': 0.0, 'unique_emoji': 0}

In [30]:
df.to_csv('clean_dynamically-generated-hate-speech-dataset.csv',index=False)